<a href="https://colab.research.google.com/github/GKCSV/Resilia_ProjetoFinal_M6/blob/GK/Analise_de_despesas_publicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ANÁLISE DE DESPESAS PÚBLICAS GERADAS POR DEPUTADOS NOS ANOS DE 2019, 2020 E 2021**

#**1. Preparação dos dados para consultas:**

##1.1. Inicialização do ambiente e importação dos dados:

In [61]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from matplotlib.ticker import StrMethodFormatter
from sklearn.preprocessing import MinMaxScaler

In [62]:
despesas2019 = pd.read_csv('https://raw.githubusercontent.com/GKCSV/Resilia_ProjetoFinal_M6/main/Ano-2019.csv', sep = ';')
despesas2020 = pd.read_csv('https://raw.githubusercontent.com/GKCSV/Resilia_ProjetoFinal_M6/main/Ano-2020.csv', sep = ';')
despesas2021 = pd.read_csv('https://raw.githubusercontent.com/GKCSV/Resilia_ProjetoFinal_M6/main/Ano-2021.csv', sep = ';')

##1.2. Formatação do dataset:

###1.2.1. Mesclagem dos DataFrames para um único:

In [63]:
despesas1920 = pd.merge(despesas2019, despesas2020, how='outer')
despesas_gerais = pd.merge(despesas1920, despesas2021, how='outer')

###1.2.2. Visualizando as colunas presentes:

In [64]:
despesas_gerais.columns.tolist()

['txNomeParlamentar',
 'cpf',
 'ideCadastro',
 'nuCarteiraParlamentar',
 'nuLegislatura',
 'sgUF',
 'sgPartido',
 'codLegislatura',
 'numSubCota',
 'txtDescricao',
 'numEspecificacaoSubCota',
 'txtDescricaoEspecificacao',
 'txtFornecedor',
 'txtCNPJCPF',
 'txtNumero',
 'indTipoDocumento',
 'datEmissao',
 'vlrDocumento',
 'vlrGlosa',
 'vlrLiquido',
 'numMes',
 'numAno',
 'numParcela',
 'txtPassageiro',
 'txtTrecho',
 'numLote',
 'numRessarcimento',
 'vlrRestituicao',
 'nuDeputadoId',
 'ideDocumento',
 'urlDocumento']

###1.2.3 Exclusão de colunas não-utilizadas através do comando .drop():
###Serão removidas as colunas:

'cpf'

'nuCarteiraParlamentar'

'nuLegislatura'

'codLegislatura'

'numSubCota'

'numEspecificacaoSubCota'

'txtNumero'

'indTipoDocumento'

'numParcela'

'txtTrecho'

'numLote'

'numRessarcimento'

In [65]:
excluir_colunas = despesas_gerais.drop(columns=['cpf', 'nuCarteiraParlamentar', 'nuLegislatura', 'codLegislatura', 'numSubCota', 'numEspecificacaoSubCota', 'txtNumero', 'indTipoDocumento', 'vlrDocumento', 'vlrGlosa', 'numParcela', 'txtTrecho', 'numLote', 'numRessarcimento', 'vlrRestituicao'])


###1.2.4. Criando cópia do dataset para atualização de nomeação:

In [66]:
despesas_selecionadas = excluir_colunas.copy()

###1.2.5 Visualizando os tipos de dados presentes nas colunas:

In [67]:
despesas_selecionadas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555500 entries, 0 to 555499
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   txNomeParlamentar          555500 non-null  object 
 1   ideCadastro                553759 non-null  float64
 2   sgUF                       553759 non-null  object 
 3   sgPartido                  553759 non-null  object 
 4   txtDescricao               555500 non-null  object 
 5   txtDescricaoEspecificacao  136177 non-null  object 
 6   txtFornecedor              555467 non-null  object 
 7   txtCNPJCPF                 531710 non-null  object 
 8   datEmissao                 539850 non-null  object 
 9   vlrLiquido                 555500 non-null  float64
 10  numMes                     555500 non-null  int64  
 11  numAno                     555500 non-null  int64  
 12  txtPassageiro              129493 non-null  object 
 13  nuDeputadoId               55

###1.2.6. Corrigindo o formato da data:

In [68]:
despesas_selecionadas['datEmissao'] = pd.to_datetime(despesas_selecionadas['datEmissao'], format = '%Y/%m/%d')

###1.2.7. Exibindo a contagem de dados:

In [69]:
despesas_selecionadas.count()

txNomeParlamentar            555500
ideCadastro                  553759
sgUF                         553759
sgPartido                    553759
txtDescricao                 555500
txtDescricaoEspecificacao    136177
txtFornecedor                555467
txtCNPJCPF                   531710
datEmissao                   539850
vlrLiquido                   555500
numMes                       555500
numAno                       555500
txtPassageiro                129493
nuDeputadoId                 555500
ideDocumento                 555500
urlDocumento                 349639
dtype: int64

###1.2.8 Criando o DataFrame do Estado de MS:

In [70]:
despesas_ms = despesas_selecionadas.loc[despesas_selecionadas['sgUF']=='MS']

###1.2.9. Criando o DataFrame das Lideranças Partidárias:

In [71]:
liderancas_partidarias = despesas_selecionadas[despesas_selecionadas['sgUF'].isnull()]

###1.2.10. Renomeando colunas:

In [72]:
despesas_ms_renomeado = despesas_ms.rename(columns={'txNomeParlamentar':'Nome-Parlamentar', 'ideCadastro':'ID-Parlamentar', 'sgUF':'Estado', 'sgPartido':'Partido', 'txtDescricao':'Despesa-Descrição', 'txtDescricaoEspecificacao':'Despesa-Especificação', 'txtFornecedor':'Nome-Fornecedor', 'txtCNPJCPF':'CNPJ/CPF-Fornecedor', 'datEmissao':'Emissão-NF', 'vlrLiquido':'Valor Liquido-NF', 'numMes':'Mês', 'numAno':'Ano', 'txtPassageiro':'Passageiro', 'nuDeputadoId':'ID-Parlamentar-Compra', 'ideDocumento':'Numero-NF', 'urlDocumento':'URL-NF'})

In [73]:
liderancas_partidarias_renomeado = liderancas_partidarias.rename(columns={'txNomeParlamentar':'Nome-Parlamentar', 'ideCadastro':'ID-Parlamentar', 'sgUF':'Estado', 'sgPartido':'Partido', 'txtDescricao':'Despesa-Descrição', 'txtDescricaoEspecificacao':'Despesa-Especificação', 'txtFornecedor':'Nome-Fornecedor', 'txtCNPJCPF':'CNPJ/CPF-Fornecedor', 'datEmissao':'Emissão-NF', 'vlrLiquido':'Valor Liquido-NF', 'numMes':'Mês', 'numAno':'Ano', 'txtPassageiro':'Passageiro', 'nuDeputadoId':'ID-Parlamentar-Compra', 'ideDocumento':'Numero-NF', 'urlDocumento':'URL-NF'})

###1.2.11. Visualização dos DataFrames para consultas:

In [74]:
despesas_ms_renomeado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7970 entries, 29077 to 553678
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Nome-Parlamentar       7970 non-null   object        
 1   ID-Parlamentar         7970 non-null   float64       
 2   Estado                 7970 non-null   object        
 3   Partido                7970 non-null   object        
 4   Despesa-Descrição      7970 non-null   object        
 5   Despesa-Especificação  2519 non-null   object        
 6   Nome-Fornecedor        7970 non-null   object        
 7   CNPJ/CPF-Fornecedor    7712 non-null   object        
 8   Emissão-NF             7724 non-null   datetime64[ns]
 9   Valor Liquido-NF       7970 non-null   float64       
 10  Mês                    7970 non-null   int64         
 11  Ano                    7970 non-null   int64         
 12  Passageiro             1562 non-null   object        
 1

In [75]:
despesas_ms_renomeado.columns.tolist()

['Nome-Parlamentar',
 'ID-Parlamentar',
 'Estado',
 'Partido',
 'Despesa-Descrição',
 'Despesa-Especificação',
 'Nome-Fornecedor',
 'CNPJ/CPF-Fornecedor',
 'Emissão-NF',
 'Valor Liquido-NF',
 'Mês',
 'Ano',
 'Passageiro',
 'ID-Parlamentar-Compra',
 'Numero-NF',
 'URL-NF']

In [76]:
liderancas_partidarias_renomeado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1741 entries, 0 to 450338
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Nome-Parlamentar       1741 non-null   object        
 1   ID-Parlamentar         0 non-null      float64       
 2   Estado                 0 non-null      object        
 3   Partido                0 non-null      object        
 4   Despesa-Descrição      1741 non-null   object        
 5   Despesa-Especificação  151 non-null    object        
 6   Nome-Fornecedor        1741 non-null   object        
 7   CNPJ/CPF-Fornecedor    1741 non-null   object        
 8   Emissão-NF             1741 non-null   datetime64[ns]
 9   Valor Liquido-NF       1741 non-null   float64       
 10  Mês                    1741 non-null   int64         
 11  Ano                    1741 non-null   int64         
 12  Passageiro             41 non-null     object        
 13  I

In [77]:
liderancas_partidarias_renomeado.columns.tolist()

['Nome-Parlamentar',
 'ID-Parlamentar',
 'Estado',
 'Partido',
 'Despesa-Descrição',
 'Despesa-Especificação',
 'Nome-Fornecedor',
 'CNPJ/CPF-Fornecedor',
 'Emissão-NF',
 'Valor Liquido-NF',
 'Mês',
 'Ano',
 'Passageiro',
 'ID-Parlamentar-Compra',
 'Numero-NF',
 'URL-NF']

#**2. Exportando os DataFrames "depesas_ms" e "liderancas_partidarias" para arquivos .xlsx:**

###**Aqui serão gerados dois arquivos .xlsx a serem utilizado no Power BI:**

In [78]:
with pd.ExcelWriter('despesas_ms.xlsx') as writer:  
    despesas_ms_renomeado.to_excel(writer, sheet_name='MS')

In [79]:
with pd.ExcelWriter('liderancas_partidarias.xlsx') as writer:  
    liderancas_partidarias_renomeado.to_excel(writer, sheet_name='MS')

#**3. Análise dos dados:**

###2.1. - Qual o maior valor que um parlamentar gastou com um único fornecedor? Este fornecedor em específico possui alguma ligação com o parlamentar identificado?

###2.2. - Qual o maior valor gasto em uma única despesa de alimentação? Existe alguma explicação plausível?

###2.3. - Qual o percentual de notas fiscais indisponíveis para consulta? Nesta relação está inclusa alguma categoria que possa vir a configurar fraude?

###2.4 - De possíveis fraudes por falta de prestação de contas, qual o top 5 parlamentares?

###2.5 - Qual o parlamentar mais caro para o governo se considerado suas despesas pessoais e as despesas a partir de lideranças partidárias?

###2.6 - Existe alguma despesa duplicada entre parlamentar x liderança partidária?